In [5]:
# =============================
# 0) 기본 라이브러리
# =============================
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import re

# -----------------------------
# 1) 데이터 준비
# -----------------------------
# 내가 실질적으로 준비하는 부분. (전처리)
# 데이터 준비와 label/feature 분리, 결측치 제거, 학습/테스트 데이터 분리.
# 이후 구성, 학습, 평가 부분은 사이킷런이 해준것.

df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/diabetes_1.csv")

# 1) feature / label 분리
X = df.drop(columns=["BMI"])
y = df["BMI"]

# 숫자형만 사용(문자형이 섞였다면 간단히 제외)
X = X.select_dtypes(include=[np.number])

# 2) 결측치 제거(디폴트 스타일 유지: dropna)
#    X와 y를 합쳐 결측이 있는 행을 통째로 제거
data = pd.concat([X, y], axis=1).dropna()
X = data.drop(columns=["BMI"])
y = data["BMI"]

# 3) 학습용 / 테스트용 분리 (랜덤 셔플 포함)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# -----------------------------
# 2) 모델 구성
# -----------------------------
dt = DecisionTreeRegressor(random_state=42)
rf = RandomForestRegressor(n_estimators=300, random_state=42)
lr = LinearRegression()
# 구성된 모델 가져옴. 분류와 다른점은 회귀이기 때문에 regressor 함수를 불러온게 다름.

# -----------------------------
# 3) 모델 학습
# -----------------------------
dt.fit(X_train, y_train)
rf.fit(X_train, y_train)
lr.fit(X_train, y_train)

# -----------------------------
# 4) 모델 평가
# -----------------------------
# 회귀 평가지표:
# - MAE: 평균 절대 오차(평균적으로 얼마나 틀렸는가)
# - RMSE: 제곱근 평균 제곱 오차(큰 오차에 더 큰 패널티)
# - R2: 결정계수(평균 예측 대비 얼마나 개선되었는가)

def evaluate(model, X_test, y_test):
    pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, pred)
    mse = mean_squared_error(y_test, pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, pred)
    return mae, rmse, r2

dt_mae, dt_rmse, dt_r2 = evaluate(dt, X_test, y_test)
rf_mae, rf_rmse, rf_r2 = evaluate(rf, X_test, y_test)
lr_mae, lr_rmse, lr_r2 = evaluate(lr, X_test, y_test)

print("=== Regression Test Metrics (BMI) ===")
print(f"Decision Tree  | MAE: {dt_mae:.4f} | RMSE: {dt_rmse:.4f} | R2: {dt_r2:.4f}")
print(f"Random Forest  | MAE: {rf_mae:.4f} | RMSE: {rf_rmse:.4f} | R2: {rf_r2:.4f}")
print(f"Linear Reg.    | MAE: {lr_mae:.4f} | RMSE: {lr_rmse:.4f} | R2: {lr_r2:.4f}")


=== Regression Test Metrics (BMI) ===
Decision Tree  | MAE: 7.2513 | RMSE: 10.2043 | R2: -0.4648
Random Forest  | MAE: 5.1743 | RMSE: 6.8293 | R2: 0.3439
Linear Reg.    | MAE: 5.2110 | RMSE: 7.2278 | R2: 0.2651
